In [1]:
#librerias
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import math
import unidecode
import re

In [2]:
#https://sinergox.xm.com.co/hdrlg/Paginas/Historicos/Historicos.aspx
#https://sinergox.xm.com.co/trpr/Paginas/Historicos/Historicos.aspx?RootFolder=%2Ftrpr%2FHistricos%2FPrecios&FolderCTID=0x012000394993FA303733428C33EC91D1DFA6DB&View=%7B5CA2173E%2D1541%2D4EC7%2D9D1C%2DE145E3DFFAE3%7D

## Descarga de datos

In [3]:
#fechas: 	'1/04/2025'  y '9/09/2025'

#Precios declarados (daily bid prices op1)
archivo = "../datos/raw/PreciodeOfertaDeclarado.csv"
df_declarado = pd.read_csv(archivo, sep=";", encoding="utf-8")
df_declarado['Planta_Configuracion'] = df_declarado['CodigoPlanta'].astype(str) + "_" + df_declarado['Configuracion'].astype(str)

#Precio de oferta de la planta en el proceso del despacho ideal, en COP/kWh (daily bid prices op2)
archivo_oferta = "../datos/raw/Preciodeofertaplanta.csv"
df_oferta = pd.read_csv(archivo_oferta, sep=",", encoding="utf-8")


#Precio de bolsa horario (daily market clearing prices)
archivo_bolsa = "../datos/raw/Preciodebolsahorario.csv"
df_bolsa = pd.read_csv(archivo_bolsa, sep=",", encoding="utf-8")

#Disponibilidad Comercial por Planta (hourly-daily available capacity)
#https://www.simem.co/datadetail/24F4EC70-2DA0-4551-A188-FA750A7D66A5

julio = pd.read_csv("../datos/raw/Disponibilidadcomercialporplantajulio.csv", sep=',', encoding='utf-8')
septiembre = pd.read_csv("../datos/raw/Disponibilidadcomercialporplantaseptiembre.csv", sep=',', encoding='utf-8')
# Concatenar verticalmente
df_capacity = pd.concat([julio, septiembre], axis=0)
# Eliminar duplicados exactos
df_capacity = df_capacity.drop_duplicates()
df_capacity = df_capacity.reset_index(drop=True)



#Pronóstico del CND de demanda por Área operativa (daily aggregate demand)
archivo_pronostico= "../datos/raw/PronósticodelCNDdedemandaporÁreaoperativa.csv"
df_pronostico = pd.read_csv(archivo_pronostico, sep=",", encoding="utf-8")
df_pronostico_agg = (
    df_pronostico.groupby("FechaHora", as_index=False)["PronDem"]
      .sum()
      .rename(columns={"PronDem": "daily_agg_demand"})
)

# Despacho programado recursos de generación (cantidades de equilibrio)
#https://www.simem.co/datadetail/FF027B64-9896-4844-B827-9DC05F690668
archivo_despacho = "../datos/raw/Despachoprogramadorecursosdegeneracion.csv"
df_despacho = pd.read_csv(archivo_despacho, sep=",", encoding="utf-8")
df_despacho_agg = (
    df_despacho.groupby("FechaHora", as_index=False)["Valor"]
      .sum()
      .rename(columns={"Valor": "daily_eq_demand"})
)


#contratos bilaterales
archivo_contratos_bilaterales = "../datos/raw/Compras_Contrato_(kWh)_2025.xlsx"
df_bilaterales = pd.read_excel(archivo_contratos_bilaterales,header=2)

# agentes
archivo_agentes = "../datos/raw/Agentes.xlsx"
df_agentes = pd.read_excel(archivo_agentes,header=4)

# agentes y plantas
archivo_plantas = "../datos/raw/Listado_Recursos_Generacion.xlsx"
df_plantas = pd.read_excel(archivo_plantas,header=3)


#Resrvas hidraulicas
#https://www.simem.co/datadetail/9605B04C-8D3A-40A9-94D3-5535670D2F9A
archivo_hidraulica = "../datos/raw/ReservasHidráulicas.csv"
df_hidraulica = pd.read_csv(archivo_hidraulica, sep=",", encoding="utf-8")

# Hidráulica (reservas)
archivo_hidraulica2 = "../datos/raw/reservas.xlsx"
df_hidraulica2 = pd.read_excel(archivo_hidraulica2, header=3)

# Ríos
archivo_rios = "../datos/raw/rios.xlsx"
df_rios = pd.read_excel(archivo_rios, header=3)

#petroleo
archivo_petroleo = "../datos/raw/petroleo.csv"
df_petroleo = pd.read_csv(archivo_petroleo, sep=",", encoding="utf-8")



In [4]:
df_despacho[df_despacho['CodigoElementoGeneracion'].isin(['GVIO', 'PES1', 'SNCR'])]

,FechaHora,CodigoDuracion,CodigoElementoGeneracion,Valor
415,2025-08-11 01:00:00,PT1H,GVIO,1245000
437,2025-08-11 01:00:00,PT1H,PES1,1180000
469,2025-08-11 01:00:00,PT1H,SNCR,0
741,2025-08-11 02:00:00,PT1H,GVIO,1245000
763,2025-08-11 02:00:00,PT1H,PES1,1180000
...,...,...,...,...
1264238,2025-04-05 22:00:00,PT1H,PES1,1180000
1264283,2025-04-05 22:00:00,PT1H,SNCR,713000
1264441,2025-04-05 23:00:00,PT1H,GVIO,0
1264472,2025-04-05 23:00:00,PT1H,PES1,1180000


### bilaterales y cantidades ofertadas

In [5]:
df_agentes

,CÓDIGO SIC,ACTIVIDAD,NOMBRE AGENTE,NIT,FECHA INICIO,Unnamed: 5
0,COGC,Comercializador,COMERCIALIZADORA 4G ENERGIA S.A.S E.S.P,901787438-8,2025-01-27 00:00:00,NaT
1,COGG,Generador,COMERCIALIZADORA 4G ENERGIA S.A.S E.S.P,901787438-8,2025-01-27 00:00:00,NaT
2,ECBG,Generador,ECOARDOBELA I S.A.S. E.S.P.,901654071-8,2025-01-29 00:00:00,NaT
3,CTCG,Generador,CENTRAL TERMOCARTAGENA S.A.S. E.S.P,901707026-5,2025-02-05 00:00:00,NaT
4,DMRC,Comercializador,DEMAND RESPONSE S.A.S. E.S.P,901823077-7,2025-02-06 00:00:00,NaT
...,...,...,...,...,...,...
57,SOCG,Generador,COMPAÑÍA ELÉCTRICA DE SOCHAGOTA S.A. E.S.P.,COMPAÑÍA ELÉCTRICA DE SOCHAGOTA S.A.S. E.S.P.,800219925-1,2025-08-15
58,XAFC,Comercializador,ABO WIND RENOVABLES PROYECTO DIECISIETE S.A.S....,GREENYELLOW JEQUES S.A.S E.S.P,901454341-3,2025-08-25
59,XAFG,Generador,ABO WIND RENOVABLES PROYECTO DIECISIETE S.A.S....,GREENYELLOW JEQUES S.A.S E.S.P,901454341-3,2025-08-25
60,EOEG,Generador,EOLOS ENERGÍA S.A.S. E.S.P.,EOLOS ENERGÍA S.A.S. E.S.P. EN LIQUIDACIÓN,900681857-0,2025-11-16


In [6]:
#crear mapa de agentes y sus plantas asociadas
def norm_agente(x):
    x = str(x).lower()                 # todo en minúscula
    x = unidecode.unidecode(x)         # quitar tildes
    x = x.replace(".", "")             # quitar puntos (para s.a.s. → sas)
    x = re.sub(r"\s+", " ", x)         # colapsar espacios múltiples
    return x.strip()

df_agentes = df_agentes.rename(columns={
    "CÓDIGO SIC": "CodigoAgente",
    "NOMBRE AGENTE": "agente"
})

df_plantas = df_plantas.rename(columns={
    "Código SIC": "CodigoPlanta",
    "Agente Representante": "agente"
})

df_agentes["agente"] = df_agentes["agente"].map(norm_agente)
df_plantas["agente"] = df_plantas["agente"].map(norm_agente)


df_agentes_plantas= df_plantas[['CodigoPlanta', 'agente']].merge(df_agentes[['agente', 'CodigoAgente']], on="agente", how="left")
 

In [7]:
#crear el ratio que proporcion de la produccion de un dia representa cada planta para cada agente
df_agentes_plantas_unique = df_agentes_plantas.drop_duplicates(subset=["CodigoPlanta"])
df_capacity_bilateral = df_capacity.merge(
    df_agentes_plantas_unique[["CodigoPlanta", "agente", "CodigoAgente"]],
    on="CodigoPlanta",
    how="left"
)

df_capacity_bilateral["ratio"] = df_capacity_bilateral.groupby(
    ["CodigoAgente", "FechaHora"]
)["Valor"].transform(lambda x: x / x.sum())

df_long = df_bilaterales.melt(
    id_vars=["Fecha", "Código Agente", "Mercado"],
    value_vars=[str(h) for h in range(24)],
    var_name="Hora",
    value_name="Valor"
)

df_long["FechaHora"] = pd.to_datetime(df_long["Fecha"]) + pd.to_timedelta(df_long["Hora"].astype(int), unit="h")

df_long = df_long[["FechaHora", "Código Agente", "Mercado", "Valor"]]
df_long = df_long.rename(columns={"Valor": "Valor_bilateral"})



In [8]:
# repartir esa proporcion con la energia de contratos bilaterales, o sea si se comprometio a producir 5kw y tiene 3 plantas con propocion 50%, 30%, 20, entonces la primera produce 5*50%, la segunda 5*30% y la tercera 5*20%
df_long["FechaHora"] = pd.to_datetime(df_long["FechaHora"])
df_capacity_bilateral["FechaHora"] = pd.to_datetime(df_capacity_bilateral["FechaHora"])

df_merged_bilateral = df_long.merge(
    df_capacity_bilateral,
    left_on=["FechaHora", "Código Agente"],
    right_on=["FechaHora", "CodigoAgente"],
    how="right"
)

df_merged_bilateral["Valor_bilateral"] = df_merged_bilateral["Valor_bilateral"].fillna(0)
df_merged_bilateral["valor_pos"] = df_merged_bilateral["Valor"] - df_merged_bilateral["Valor_bilateral"]


df_merged_bilateral

,FechaHora,Código Agente,Mercado,Valor_bilateral,CodigoVariable,CodigoDuracion,UnidadMedida,Version,Valor,CodigoPlanta,agente,CodigoAgente,ratio,valor_pos
0,2025-05-25 16:00:00,NaN,NaN,0.0,DispCom,PT1H,kWh,TX3,148.56,3ACC,celsia colombia sa esp,NaN,NaN,148.56
1,2025-05-25 15:00:00,NaN,NaN,0.0,DispCom,PT1H,kWh,TX3,2706.61,3ACC,celsia colombia sa esp,NaN,NaN,2706.61
2,2025-05-25 14:00:00,NaN,NaN,0.0,DispCom,PT1H,kWh,TX3,3574.43,3ACC,celsia colombia sa esp,NaN,NaN,3574.43
3,2025-05-25 13:00:00,NaN,NaN,0.0,DispCom,PT1H,kWh,TX3,3642.49,3ACC,celsia colombia sa esp,NaN,NaN,3642.49
4,2025-05-25 12:00:00,NaN,NaN,0.0,DispCom,PT1H,kWh,TX3,5078.19,3ACC,celsia colombia sa esp,NaN,NaN,5078.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2946659,2025-08-10 04:00:00,NaN,NaN,0.0,DispCom,PT1H,kWh,TX1,2825.03,VNTB,hidrotolima sa esp,NaN,NaN,2825.03
2946660,2025-08-10 03:00:00,NaN,NaN,0.0,DispCom,PT1H,kWh,TX1,2842.82,VNTB,hidrotolima sa esp,NaN,NaN,2842.82
2946661,2025-08-10 02:00:00,NaN,NaN,0.0,DispCom,PT1H,kWh,TX1,2855.33,VNTB,hidrotolima sa esp,NaN,NaN,2855.33
2946662,2025-08-10 01:00:00,NaN,NaN,0.0,DispCom,PT1H,kWh,TX1,2869.57,VNTB,hidrotolima sa esp,NaN,NaN,2869.57


In [9]:
df_capacity["FechaHora"] = pd.to_datetime(df_capacity["FechaHora"])
df_merged_bilateral["FechaHora"] = pd.to_datetime(df_merged_bilateral["FechaHora"])

df_capacity = df_capacity.merge(
    df_merged_bilateral[["FechaHora", "Valor", "CodigoPlanta", "valor_pos"]],
    on=["FechaHora", "Valor", "CodigoPlanta"],
    how="left"
)

df_capacity["valor_pos"] = df_capacity["valor_pos"].clip(lower=0)
df_capacity["valor_pos"] = pd.to_numeric(df_capacity["valor_pos"], errors="coerce").fillna(0)
#df_capacity["valor_pos"] = df_capacity["valor_pos"].fillna(0)


### Covariables x

In [10]:
# df rios
df_rios = df_rios[['Fecha', 'Aportes %', 'Nombre Río']]
df_rios = df_rios[df_rios["Nombre Río"] != "FLORIDA II"].copy()
df_rios_agg = (
    df_rios.groupby('Fecha')['Aportes %']
           .apply(list)     # crea un array de aportes por fecha
           .reset_index()
           .rename(columns={'Aportes %': 'Aportes_array'})
)

#df petroleo
df_petroleo = df_petroleo[['Fecha', 'Apertura']]


In [11]:
df_rios_agg["len"] = df_rios_agg["Aportes_array"].apply(len)
df_rios_agg["len"].value_counts()

len
43    317
Name: count, dtype: int64

In [12]:

#df hidraulico
df_hidraulica = df_hidraulica[['Fecha', 'CodigoEmbalse','VolumenUtilDiarioMasa', 'VolumenTotalMasa']]
df_hidraulica['ratio_util_total'] = (
    df_hidraulica['VolumenUtilDiarioMasa'] / df_hidraulica['VolumenTotalMasa'])

df_wide = df_hidraulica.pivot(index="Fecha", columns="CodigoEmbalse", values="ratio_util_total")
df_wide = df_wide.sort_index()
df_wide = df_wide.ffill().bfill()
df_hidraulica_completo = (
    df_wide
    .reset_index()
    .melt(id_vars="Fecha", var_name="CodigoEmbalse", value_name="ratio_util_total")
    .sort_values(["Fecha", "CodigoEmbalse"])
    .reset_index(drop=True)
)


df_hidraulica_agg = (
    df_hidraulica_completo.groupby('Fecha')['ratio_util_total']
                 .apply(list)     # lista de ratios por fecha
                 .reset_index()
                 .rename(columns={'ratio_util_total': 'Ratio_array'})
)


In [13]:
df_hidraulica_agg["len"] = df_hidraulica_agg["Ratio_array"].apply(len)
df_hidraulica_agg["len"].value_counts()

len
27    162
Name: count, dtype: int64

In [14]:
#consolidar merge  
dfs = [df_petroleo, df_rios_agg, df_hidraulica_agg]
for df in dfs:
    df["Fecha"] = pd.to_datetime(df["Fecha"], format="mixed", dayfirst=True, errors="raise")

start = pd.Timestamp("2025-04-01")
end   = pd.Timestamp("2025-09-09")

dfs_f = [df[(df["Fecha"] >= start) & (df["Fecha"] <= end)].copy() for df in dfs]

df_covariates = reduce(
    lambda left, right: pd.merge(left, right, on="Fecha", how="inner"),
    dfs_f
)

df_covariates["covariates"] = df_covariates.apply(
    lambda row: [row["Apertura"]] + row["Aportes_array"] + row["Ratio_array"],
    axis=1
)
df_covariates=df_covariates[['Fecha', 'covariates']]


## Clusterizacion

In [15]:
def limpiar_array(arr):
    clean = []
    for x in arr:
        if isinstance(x, str):
            x = x.replace(",", ".")
            try:
                clean.append(float(x))
            except:
                clean.append(np.nan)
        else:
            clean.append(float(x) if not math.isnan(x) else np.nan)

    clean = np.array(clean, dtype=float)

    mask = np.isnan(clean)
    if mask.any():
        clean[mask] = np.nanmean(clean)

    return clean

df_covariates["covariates"] = df_covariates["covariates"].apply(limpiar_array)


X = np.stack(df_covariates["covariates"].values)



In [16]:



K_range = range(2, 30)
scores = []

for k in K_range:
    km = KMeans(n_clusters=k, n_init=20, random_state=123)
    labels = km.fit_predict(X)
    score = silhouette_score(X, labels)
    scores.append((k, score))
    print(f"k = {k:2d} | silhouette = {score:.4f}")

best_k, best_score = max(scores, key=lambda x: x[1])
print("\nMejor k:", best_k, "| score:", best_score)


k =  2 | silhouette = 0.2585
k =  3 | silhouette = 0.2817
k =  4 | silhouette = 0.1774
k =  5 | silhouette = 0.1854
k =  6 | silhouette = 0.1315
k =  7 | silhouette = 0.2029
k =  8 | silhouette = 0.1928
k =  9 | silhouette = 0.1443
k = 10 | silhouette = 0.1855
k = 11 | silhouette = 0.1434
k = 12 | silhouette = 0.1501
k = 13 | silhouette = 0.1362
k = 14 | silhouette = 0.1464
k = 15 | silhouette = 0.1420
k = 16 | silhouette = 0.1301
k = 17 | silhouette = 0.1507
k = 18 | silhouette = 0.1331
k = 19 | silhouette = 0.1215
k = 20 | silhouette = 0.1244
k = 21 | silhouette = 0.1459
k = 22 | silhouette = 0.1246
k = 23 | silhouette = 0.0970
k = 24 | silhouette = 0.1291
k = 25 | silhouette = 0.1414
k = 26 | silhouette = 0.1185
k = 27 | silhouette = 0.1344
k = 28 | silhouette = 0.1112
k = 29 | silhouette = 0.1182

Mejor k: 3 | score: 0.2817229706438937


In [17]:

best_k = 3 #K_range[np.argmax(scores)]

km_final = KMeans(n_clusters=best_k, n_init=50, random_state=123)
df_covariates["cluster_xs"] = km_final.fit_predict(X)


In [18]:
df_covariates["Fecha"] = pd.to_datetime(df_covariates["Fecha"])
df_covariates["weekday"] = df_covariates["Fecha"].dt.day_name()

In [19]:
# Crear la llave combinada weekday + cluster_x
df_covariates["cluster_key"] = (
    df_covariates["weekday"].astype(str) + "_" +
    df_covariates["cluster_xs"].astype(str)
)

# Convertir cada combinación única en un número entero
df_covariates["cluster"] = df_covariates["cluster_key"].astype("category").cat.codes


## Merges

In [20]:
df_oferta

,CodigoVariable,FechaHora,CodigoDuracion,UnidadMedida,CodigoPlanta,Version,Valor
0,POf_Original,2025-05-25 23:00:00,P1D,COP/kWh,TEC1,TXF,1486.46
1,POf_Original,2025-05-25 22:00:00,P1D,COP/kWh,TEC1,TXF,1486.46
2,POf_Original,2025-05-25 21:00:00,P1D,COP/kWh,TEC1,TXF,1486.46
3,POf_Original,2025-05-25 20:00:00,P1D,COP/kWh,TEC1,TXF,1486.46
4,POf_Original,2025-05-25 19:00:00,P1D,COP/kWh,TEC1,TXF,1486.46
...,...,...,...,...,...,...,...
670603,POf_Original,2025-07-13 04:00:00,P1D,COP/kWh,ZPA2,TX4,379.71
670604,POf_Original,2025-07-13 03:00:00,P1D,COP/kWh,ZPA2,TX4,379.71
670605,POf_Original,2025-07-13 02:00:00,P1D,COP/kWh,ZPA2,TX4,379.71
670606,POf_Original,2025-07-13 01:00:00,P1D,COP/kWh,ZPA2,TX4,379.71


In [21]:
# Renombrar columnas antes del merge

df_oferta_renamed = df_oferta[['FechaHora', 'CodigoPlanta', 'Valor']].rename(columns={'Valor': 'precio'})
df_capacity_renamed = df_capacity[['FechaHora','Valor', 'CodigoPlanta', 'valor_pos']].rename(columns={'Valor': 'cantidad', 'valor_pos':'cantidad_pos'}) #ya incorpora contratos bilaterales

df_oferta_renamed['FechaHora'] = pd.to_datetime(df_oferta_renamed['FechaHora'])
df_capacity_renamed['FechaHora'] = pd.to_datetime(df_capacity_renamed['FechaHora'])

df_merged = df_oferta_renamed.merge(
    df_capacity_renamed,
    on=['FechaHora', 'CodigoPlanta'],
    how='left'
)

# Reemplazar NaN en cantidad por 0
df_merged['cantidad'] = df_merged['cantidad'].fillna(0)
df_merged = df_merged.drop_duplicates(subset=['FechaHora', 'CodigoPlanta'], keep='first')

In [22]:
df_merged['Fecha'] = pd.to_datetime(df_merged['FechaHora']).dt.date
df_covariates['Fecha'] = pd.to_datetime(df_covariates['Fecha']).dt.date

df_final = pd.merge(
    df_merged,
    df_covariates[['Fecha', 'cluster']],
    on='Fecha',
    how='inner'
)
df_final

,FechaHora,CodigoPlanta,precio,cantidad,cantidad_pos,Fecha,cluster
0,2025-05-25 23:00:00,TEC1,1486.46,213000.0,213000.0,2025-05-25,6
1,2025-05-25 22:00:00,TEC1,1486.46,213000.0,213000.0,2025-05-25,6
2,2025-05-25 21:00:00,TEC1,1486.46,213000.0,213000.0,2025-05-25,6
3,2025-05-25 20:00:00,TEC1,1486.46,213000.0,213000.0,2025-05-25,6
4,2025-05-25 19:00:00,TEC1,1486.46,213000.0,213000.0,2025-05-25,6
...,...,...,...,...,...,...,...
221827,2025-09-05 04:00:00,ZPA2,372.94,36000.0,36000.0,2025-09-05,2
221828,2025-09-05 03:00:00,ZPA2,372.94,36000.0,36000.0,2025-09-05,2
221829,2025-09-05 02:00:00,ZPA2,372.94,36000.0,36000.0,2025-09-05,2
221830,2025-09-05 01:00:00,ZPA2,372.94,36000.0,36000.0,2025-09-05,2


In [23]:
#transado en el cierre

df_transado = pd.merge(
    df_bolsa[['FechaHora', 'Valor']],
    df_despacho_agg[['FechaHora', 'daily_eq_demand']],
    on='FechaHora',
    how='left'
)
df_transado = df_transado.rename(columns={
    'Valor': 'precio',
    'daily_eq_demand': 'demanda'
})

In [24]:
df_despacho_agg[df_despacho_agg['FechaHora']=='2025-05-25 23:00:00']

,FechaHora,daily_eq_demand
1319,2025-05-25 23:00:00,8524260


In [25]:
df_bolsa[df_bolsa['FechaHora']=='2025-05-25 23:00:00']

,CodigoVariable,FechaHora,CodigoDuracion,UnidadMedida,Version,Valor
49535,PB_Int,2025-05-25 23:00:00,PT1H,COP/kWh,TX1,103.00
49559,PB_Nal,2025-05-25 23:00:00,PT1H,COP/kWh,TX1,108.56
49583,PB_Tie,2025-05-25 23:00:00,PT1H,COP/kWh,TX1,103.00
49607,PB_Int,2025-05-25 23:00:00,PT1H,COP/kWh,TXF,105.39
49631,PB_Nal,2025-05-25 23:00:00,PT1H,COP/kWh,TXF,111.04
49655,PB_Tie,2025-05-25 23:00:00,PT1H,COP/kWh,TXF,105.39
49679,PB_Int,2025-05-25 23:00:00,PT1H,COP/kWh,TX2,103.00
49703,PB_Nal,2025-05-25 23:00:00,PT1H,COP/kWh,TX2,106.56
49727,PB_Tie,2025-05-25 23:00:00,PT1H,COP/kWh,TX2,103.00
49751,PB_Int,2025-05-25 23:00:00,PT1H,COP/kWh,TXR,105.39


In [40]:
# Asegurarse de que las columnas 'FechaHora' sean datetime
df_final['FechaHora'] = pd.to_datetime(df_final['FechaHora'])
df_transado['FechaHora'] = pd.to_datetime(df_transado['FechaHora'])

# Filtrar df_transado: solo fechas que aparecen en df_final
df_transado_filtrado = df_transado[df_transado['FechaHora'].isin(df_final['FechaHora'])]

# Filtrar df_final: solo fechas que aparecen en df_transado_filtrado
df_final_filtrado = df_final[df_final['FechaHora'].isin(df_transado_filtrado['FechaHora'])]
df_final_filtrado["cantidad_pos"] = df_final_filtrado["cantidad_pos"].fillna(0)


### Crear gamma para cada firma

In [41]:
df = df_final_filtrado.copy()

agg = (
    df.groupby('CodigoPlanta')
      .agg(
          Q_i=('cantidad_pos', 'sum'),
          p_i=('precio', 'median')
      )
      .reset_index()
)

eps = 1e-12
Qmax = agg['Q_i'].max()
pmax = agg['p_i'].max()

size_factor = np.sqrt(agg['Q_i'] / (Qmax + eps))
price_factor = (pmax + eps) / (agg['p_i'] + eps)

agg['gamma_thumb'] = size_factor * price_factor

df_final_filtrado = df.merge(
    agg[['CodigoPlanta', 'gamma_thumb']],
    on='CodigoPlanta',
    how='left'
)


In [42]:
df_final_filtrado['gamma_thumb'].describe()

count    221832.000000
mean          5.572205
std           6.517271
min           0.000000
25%           1.215117
50%           3.134335
75%           6.684364
max          25.654511
Name: gamma_thumb, dtype: float64

In [43]:
df_final_filtrado[df_final_filtrado['gamma_thumb'] == df_final_filtrado['gamma_thumb'].max()]


,FechaHora,CodigoPlanta,precio,cantidad,cantidad_pos,Fecha,cluster,gamma_thumb
1488,2025-05-25 23:00:00,PES1,102.64,1180000.0,1180000.0,2025-05-25,6,25.654511
1489,2025-05-25 22:00:00,PES1,102.64,1180000.0,1180000.0,2025-05-25,6,25.654511
1490,2025-05-25 21:00:00,PES1,102.64,1180000.0,1180000.0,2025-05-25,6,25.654511
1491,2025-05-25 20:00:00,PES1,102.64,1180000.0,1180000.0,2025-05-25,6,25.654511
1492,2025-05-25 19:00:00,PES1,102.64,1180000.0,1180000.0,2025-05-25,6,25.654511
...,...,...,...,...,...,...,...,...
220027,2025-09-05 04:00:00,PES1,95.00,1140000.0,1140000.0,2025-09-05,2,25.654511
220028,2025-09-05 03:00:00,PES1,95.00,1140000.0,1140000.0,2025-09-05,2,25.654511
220029,2025-09-05 02:00:00,PES1,95.00,1140000.0,1140000.0,2025-09-05,2,25.654511
220030,2025-09-05 01:00:00,PES1,95.00,1140000.0,1140000.0,2025-09-05,2,25.654511


### Guardar resultados

In [44]:
df_final_filtrado.to_csv("../datos/procesado/df_final.csv", index=False)
df_transado_filtrado.to_csv("../datos/procesado/df_transado.csv", index=False)
df_despacho_agg.to_csv("../datos/procesado/df_despacho_agg.csv", index=False)

In [45]:
df_transado_filtrado['FechaHora'].min()

Timestamp('2025-04-01 00:00:00')

In [46]:
df_transado_filtrado['FechaHora'].max()

Timestamp('2025-09-09 23:00:00')